# LMNA

In [1]:
import genophenocorr
print(f"Using genophenocorr version {genophenocorr.__version__}")

Using genophenocorr version 0.1.1dev


In [2]:
import hpotk

fpath_hpo = 'https://github.com/obophenotype/human-phenotype-ontology/releases/download/v2024-02-08/hp.json'
hpo = hpotk.load_minimal_ontology(fpath_hpo)

In [3]:
from genophenocorr.preprocessing import configure_caching_cohort_creator, load_phenopacket_folder

fpath_phenopackets = 'input/'
cohort_creator = configure_caching_cohort_creator(hpo)
cohort = load_phenopacket_folder(fpath_phenopackets, cohort_creator)

Patients Created: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 127/127 [00:00<00:00, 208.00it/s]
Validated under none policy
No errors or warnings were found


## Transcript
Get it by looking for gene symbol in https://www.ncbi.nlm.nih.gov/clinvar/ look for pathogenic
zcopy it and go to https://variantvalidator.org/service/validate/, then scroll down to copy GRCH38 string, copy this and put into same website

looking for MANE select plus clinical (most important transcript for clinic) if doesnt exist, use clinvar result

In [4]:
tx_id = 'NM_170707.4'

## Explore Cohort

In [5]:
from IPython.display import HTML, display
from genophenocorr.view import CohortViewer

viewer = CohortViewer(hpo)

In [6]:
display(HTML(viewer.cohort_summary_table(cohort)))

Item,Description
Total Individuals,127
Total Unique HPO Terms,214
Total Unique Variants,127


In [7]:
cohort.all_diseases

{Disease(identifier=OMIM:115200, name=Cardiomyopathy, dilated, 1A, is_present=True),
 Disease(identifier=OMIM:151660, name=Lipodystrophy, familial partial, type 2, is_present=True),
 Disease(identifier=OMIM:176670, name=Hutchinson-Gilford progeria, is_present=True),
 Disease(identifier=OMIM:181350, name=Emery-Dreifuss muscular dystrophy 2, autosomal dominant, is_present=True),
 Disease(identifier=OMIM:613205, name=LMNA-related congenital muscular dystrophy, is_present=True)}

In [8]:
display(HTML(viewer.hpo_term_counts_table(cohort))) ## Add Labels to output

HPO Term,Count
Ankle contracture (HP:0034677),39
Spinal rigidity (HP:0003306),37
Elbow contracture (HP:0034391),37
Stiff neck (HP:0025258),36
Proximal muscle weakness in upper limbs (HP:0008997),35
Dilated cardiomyopathy (HP:0001644),27
Atrial fibrillation (HP:0005110),25
Arrhythmia (HP:0011675),23
Lipodystrophy (HP:0009125),21
Distal lower limb muscle weakness (HP:0009053),21


In [9]:
display(HTML(viewer.variants_table(cohort, tx_id))) 

Variant,Effect,Count,Key
c.1824C>T,SYNONYMOUS_VARIANT,15,1_156138613_156138613_C_T
c.585C>A,MISSENSE_VARIANT,11,1_156134474_156134474_C_A
c.1580G>C,MISSENSE_VARIANT,8,1_156137204_156137204_G_C
c.608A>G,MISSENSE_VARIANT,8,1_156134497_156134497_A_G
c.1698+13C>A,INTRON_VARIANT,8,1_156137756_156137756_C_A
c.178C>G,MISSENSE_VARIANT,7,1_156115096_156115096_C_G
c.1444C>T,MISSENSE_VARIANT,7,1_156136984_156136984_C_T
c.1357C>T,MISSENSE_VARIANT,6,1_156136413_156136413_C_T
c.1072G>A,MISSENSE_VARIANT,6,1_156136036_156136036_G_A
c.254T>G,MISSENSE_VARIANT,5,1_156115172_156115172_T_G


## Configure the analysis

In [10]:
from genophenocorr.analysis import configure_cohort_analysis, CohortAnalysisConfiguration

analysis_config = CohortAnalysisConfiguration()
analysis_config.missing_implies_excluded = True
analysis_config.pval_correction = 'fdr_bh'

analysis = configure_cohort_analysis(cohort, hpo, config=analysis_config)


In [11]:
from genophenocorr.model import VariantEffect
from genophenocorr.analysis.predicate import PatientCategories
from genophenocorr.analysis.predicate.genotype import VariantEffectPredicate

by_missense = analysis.compare_by_variant_effect(VariantEffect.MISSENSE_VARIANT, tx_id=tx_id)
by_missense.summarize(hpo, PatientCategories.YES)

MISSENSE_VARIANT on NM_170707.4                       Yes             No  \
                                                    Count Percent  Count   
Lipodystrophy [HP:0009125]                          10/75     13%  15/35   
Micrognathia [HP:0000347]                             0/3      0%  15/15   
Failure to thrive [HP:0001508]                        0/3      0%  15/15   
Reduced subcutaneous adipose tissue [HP:0003758]      0/3      0%  15/15   
Proptosis [HP:0000520]                                0/3      0%  15/15   
...                                                   ...     ...    ...   
Abnormality of the face [HP:0000271]                  4/4    100%  15/15   
Abnormal electrophysiology of sinoatrial node o...    2/2    100%    2/2   
Peripheral cyanosis [HP:0034033]                      0/0      0%  13/13   
Abnormality of hepatobiliary system physiology ...    1/1    100%    0/0   
Tooth agenesis [HP:0009804]                           0/0      0%    7/7   

MISSENSE_VARIANT on NM_170707.4                                       \
                                                   Percent   p value   
Lipodystrophy [HP:0009125]                             43%  0.001167   
Micrognathia [HP:0000347]                             100%  0.001225   
Failure to thrive [HP:0001508]                        100%  0.001225   
Reduced subcutaneous adipose tissue [HP:0003758]      100%  0.001225   
Proptosis [HP:0000520]                                100%  0.001225   
...                                                    ...       ...   
Abnormality of the face [HP:0000271]                  100%  1.000000   
Abnormal electrophysiology of sinoatrial node o...    100%  1.000000   
Peripheral cyanosis [HP:0034033]                      100%  1.000000   
Abnormality of hepatobiliary system physiology ...      0%  1.000000   
Tooth agenesis [HP:0009804]                           100%  1.000000   

MISSENSE_VARIANT on NM_170707.4                                       
                                                   Corrected p value  
Lipodystrophy [HP:0009125]                                  0.079657  
Micrognathia [HP:0000347]                                   0.079657  
Failure to thrive [HP:0001508]                              0.079657  
Reduced subcutaneous adipose tissue [HP:0003758]            0.079657  
Proptosis [HP:0000520]                                      0.079657  
...                                                              ...  
Abnormality of the face [HP:0000271]                        1.000000  
Abnormal electrophysiology of sinoatrial node o...          1.000000  
Peripheral cyanosis [HP:0034033]                            1.000000  
Abnormality of hepatobiliary system physiology ...          1.000000  
Tooth agenesis [HP:0009804]                                 1.000000  

[455 rows x 6 columns]

In [12]:
by_dis = analysis.compare_disease_vs_genotype(VariantEffectPredicate(transcript_id=tx_id, effect=VariantEffect.MISSENSE_VARIANT))
by_dis.summarize(hpo, PatientCategories.YES)

MISSENSE_VARIANT on NM_170707.4    Yes             No                        \
                                 Count Percent  Count Percent       p value   
OMIM:176670                       0/90      0%  15/37     41%  8.025624e-10   
OMIM:151660                      12/90     13%   0/37      0%  1.807470e-02   
OMIM:181350                      34/90     38%   7/37     19%  5.904112e-02   
OMIM:613205                      13/90     14%   2/37      5%  2.276691e-01   
OMIM:115200                      31/90     34%  13/37     35%  1.000000e+00   

MISSENSE_VARIANT on NM_170707.4                    
                                Corrected p value  
OMIM:176670                          4.012812e-09  
OMIM:151660                          4.518674e-02  
OMIM:181350                          9.840187e-02  
OMIM:613205                          2.845863e-01  
OMIM:115200                          1.000000e+00